In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Thyroid_Cancer"
cohort = "GSE104005"

# Input paths
in_trait_dir = "../../input/GEO/Thyroid_Cancer"
in_cohort_dir = "../../input/GEO/Thyroid_Cancer/GSE104005"

# Output paths
out_data_file = "../../output/preprocess/Thyroid_Cancer/GSE104005.csv"
out_gene_data_file = "../../output/preprocess/Thyroid_Cancer/gene_data/GSE104005.csv"
out_clinical_data_file = "../../output/preprocess/Thyroid_Cancer/clinical_data/GSE104005.csv"
json_path = "../../output/preprocess/Thyroid_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [ ]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
# 1. Gene Expression Data Availability 
# Based on the series overall design, this dataset contains gene expression data using HumanHT-12 WG-DASL V4.0 R2 expression beadchip
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# Trait data is available in row 0 - disease status
trait_row = 0
# Age data is available in row 2
age_row = 2
# Gender data is available in row 3 as Sex
gender_row = 3

# 2.2 Data Type Conversion Functions
def convert_trait(value):
    """Convert thyroid cancer status to binary: 1 for cancer, 0 for non-neoplastic."""
    if value is None:
        return None
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    if 'Non-neoplastic' in value:
        return 0  # Control
    elif 'carcinoma' in value or 'Thyroid_carcinoma' in value:
        return 1  # Case
    return None

def convert_age(value):
    """Convert age to continuous numeric value."""
    if value is None:
        return None
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    try:
        return float(value)
    except ValueError:
        return None

def convert_gender(value):
    """Convert gender to binary: 0 for female, 1 for male."""
    if value is None:
        return None
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    if value.upper() == 'F':
        return 0  # Female
    elif value.upper() == 'M':
        return 1  # Male
    return None

# 3. Save Metadata
# Determine if trait data is available
is_trait_available = trait_row is not None
validate_and_save_cohort_info(is_final=False, cohort=cohort, info_path=json_path, 
                              is_gene_available=is_gene_available, 
                              is_trait_available=is_trait_available)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Create a DataFrame from the Sample Characteristics Dictionary
    # We need to first transpose the data to make it compatible
    sample_chars = {
        0: ['disease: Thyroid_carcinoma', 'disease: Non-neoplastic_thyroid'], 
        1: ['histology: PDTC', 'histology: PDTC+ATC', 'histology: PTC', 'histology: Non-neoplastic_thyroid', 
            'histology: PDTC+PTC', 'histology: PTC_lymph_node_metastasis', 'histology: PTC+PDTC'], 
        2: ['age: 74', 'age: 67', 'age: 72', 'age: 38', 'age: 50', 'age: 41', 'age: 51', 'age: 73', 
            'age: 52', 'age: 48', 'age: 59', 'age: 58', 'age: 39', 'age: 37', 'age: 33', 'age: 36', 
            'age: 70', 'age: 26', 'age: 46', 'age: 57', 'age: 44', 'age: 35', 'age: 42', 'age: 61', 'age: 49'], 
        3: ['Sex: M', 'Sex: F']
    }
    
    # Convert the dictionary to a DataFrame
    clinical_data = pd.DataFrame.from_dict(sample_chars, orient='index')
    
    # Extract clinical features
    clinical_df = geo_select_clinical_features(
        clinical_data, 
        trait=trait, 
        trait_row=trait_row, 
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the dataframe
    preview = preview_df(clinical_df)
    print("Clinical data preview:")
    print(preview)
    
    # Create the directory if it doesn't exist
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    
    # Save clinical data to CSV
    clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


### Step 3: Gene Data Extraction

In [ ]:
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"SOFT file: {soft_file}")
print(f"Matrix file: {matrix_file}")

# Set gene availability flag
is_gene_available = True  # Initially assume gene data is available

# First check if the matrix file contains the expected marker
found_marker = False
try:
    with gzip.open(matrix_file, 'rt') as file:
        for line in file:
            if "!series_matrix_table_begin" in line:
                found_marker = True
                break
    
    if found_marker:
        print("Found the matrix table marker in the file.")
    else:
        print("Warning: Could not find '!series_matrix_table_begin' marker in the file.")
        
    # Try to extract gene data from the matrix file
    gene_data = get_genetic_data(matrix_file)
    
    if gene_data.shape[0] == 0:
        print("Warning: Extracted gene data has 0 rows.")
        is_gene_available = False
    else:
        print(f"Gene data shape: {gene_data.shape}")
        # Print the first 20 gene/probe identifiers
        print("First 20 gene/probe identifiers:")
        print(gene_data.index[:20].tolist())
        
except Exception as e:
    print(f"Error extracting gene data: {e}")
    is_gene_available = False
    
    # Try to diagnose the file format
    print("Examining file content to diagnose the issue:")
    try:
        with gzip.open(matrix_file, 'rt') as file:
            for i, line in enumerate(file):
                if i < 10:  # Print first 10 lines to diagnose
                    print(f"Line {i}: {line.strip()[:100]}...")  # Print first 100 chars of each line
                else:
                    break
    except Exception as e2:
        print(f"Error examining file: {e2}")

if not is_gene_available:
    print("Gene expression data could not be successfully extracted from this dataset.")


### Step 4: Gene Identifier Review

In [ ]:
# Analyze the gene identifiers starting with "ILMN_"
# These are Illumina BeadArray probe IDs, not human gene symbols
# They need to be mapped to standard gene symbols for analysis

requires_gene_mapping = True


### Step 5: Gene Annotation

In [ ]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
gene_annotation = get_gene_annotation(soft_file)

# 2. Analyze the gene annotation dataframe to identify which columns contain the gene identifiers and gene symbols
print("\nGene annotation preview:")
print(f"Columns in gene annotation: {gene_annotation.columns.tolist()}")
print(preview_df(gene_annotation, n=5))

# Get a more complete view to understand the annotation structure
print("\nComplete sample of a few rows:")
print(gene_annotation.iloc[:3, :4].to_string())  # Show only first few columns for readability

# Examine the Symbol column which contains gene information
print("\nChecking for gene information in Symbol column:")
if 'Symbol' in gene_annotation.columns:
    sample_symbols = gene_annotation['Symbol'].head(5).tolist()
    print(f"Sample Symbol values: {sample_symbols}")
    
    # Use the library function to create the mapping
    mapping_data = get_gene_mapping(gene_annotation, 'ID', 'Symbol')
    
    # Print sample of the mapping to confirm
    print("\nSample of probe ID to gene symbol mappings:")
    print(mapping_data.head(10))
    
    # Check the size of the mapping data
    print(f"\nTotal number of probe-to-gene mappings: {len(mapping_data)}")
    
    # Check how many unique gene symbols we have
    unique_genes = mapping_data['Gene'].nunique()
    print(f"Number of unique gene symbols: {unique_genes}")
else:
    print("\nError: Could not find 'Symbol' column in the annotation data")


### Step 6: Gene Identifier Mapping

In [ ]:
# 1. Analyze gene identifiers in gene expression data and gene annotation data
# In the gene expression data, the identifiers start with ILMN_ (like ILMN_1343291)
# In the gene annotation data, these are in the 'ID' column
# Gene symbols are in the 'Symbol' column

# 2. Extract gene mapping from annotation data
# We need to map from probe IDs to gene symbols
mapping_df = get_gene_mapping(gene_annotation, 'ID', 'Symbol')
print(f"Created mapping dataframe with {len(mapping_df)} rows")

# 3. Apply gene mapping to convert probe-level measurements to gene expression data
# This handles the many-to-many relation between probes and genes as specified
gene_data = apply_gene_mapping(gene_data, mapping_df)
print(f"Mapped gene expression data shape: {gene_data.shape}")
print("First 10 gene symbols in the mapped data:")
print(gene_data.index[:10].tolist())

# To check if we have actual human gene symbols
human_genes = gene_data.index[gene_data.index.str.match(r'[A-Z0-9]{1,10}')].tolist()[:10]
print(f"\nSample human gene symbols: {human_genes}")

# Save the gene expression data to CSV
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


### Step 7: Data Normalization and Linking

In [ ]:
```python
# 1. Normalize gene symbols in the gene expression data
try:
    # Make sure the directory exists
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    
    # Use the gene_data variable from the previous step (don't try to load it from file)
    print(f"Gene data shape before normalization: {gene_data.shape}")
    
    # Apply normalization to gene symbols
    normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
    print(f"Gene data shape after normalization: {normalized_gene_data.shape}")
    
    # Save the normalized gene data
    normalized_gene_data.to_csv(out_gene_data_file)
    print(f"Normalized gene data saved to {out_gene_data_file}")
    
    # Use the normalized data for further processing
    gene_data = normalized_gene_data
    is_gene_available = True
except Exception as e:
    print(f"Error normalizing gene data: {e}")
    is_gene_available = False

# 2. Load clinical data
try:
    # Load the clinical data from file
    clinical_data = pd.read_csv(out_clinical_data_file)
    print(f"Loaded clinical data shape: {clinical_data.shape}")
    is_trait_available = True
except Exception as e:
    print(f"Error loading clinical data: {e}")
    # Reload the clinical data directly from the matrix file
    soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
    background_info, clinical_data = get_background_and_clinical_data(matrix_file)
    
    # Extract clinical features with correct indices from Step 2
    clinical_features = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=0,       # Correct trait row from Step 2
        convert_trait=convert_trait,
        gender_row=3,      # Correct gender row from Step 2
        convert_gender=convert_gender,
        age_row=2,         # Age row from Step 2
        convert_age=convert_age
    )
    
    print(f"Re-extracted clinical data shape: {clinical_features.shape}")
    print("Preview of clinical data (first 5 samples):")
    print(clinical_features.head())
    
    # Save the properly extracted clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_features.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")
    
    clinical_data = clinical_features
    is_trait_available = True

# 3. Link clinical and genetic data if both are available
if is_trait_available and is_gene_available:
    try:
        # Transpose the clinical data if needed to have the same format as gene data
        if isinstance(clinical_data, pd.DataFrame) and clinical_data.shape[0] <= 3:
            clinical_data = clinical_data.T
            clinical_data.columns = [trait, 'Age', 'Gender']
        
        # Debug column comparison to ensure compatibility
        gene_sample_ids = set(gene_data.columns)
        clinical_sample_ids = set(clinical_data.index)
        common_samples = gene_sample_ids.intersection(clinical_sample_ids)
        print(f"Found {len(common_samples)} common samples between gene and clinical data")
        
        if len(common_samples) > 0:
            # Select only common samples from both datasets
            gene_data_filtered = gene_data[list(common_samples)]
            clinical_data_filtered = clinical_data.loc[list(common_samples)]
            
            # Create linked data
            linked_data = pd.DataFrame(index=clinical_data_filtered.index)
            
            # Add clinical features
            if trait in clinical_data_filtered.columns:
                linked_data[trait] = clinical_data_filtered[trait]
            if 'Age' in clinical_data_filtered.columns:
                linked_data['Age'] = clinical_data_filtered['Age']
            if 'Gender' in clinical_data_filtered.columns:
                linked_data['Gender'] = clinical_data_filtered['Gender']
            
            # Add gene features - transpose so genes are columns
            gene_data_filtered_T = gene_data_filtered.T
            for gene in gene_data_filtered.index:
                linked_data[gene] = gene_data_filtered_T[gene]
                
            print(f"Initial linked data shape: {linked_data.shape}")
            print("Preview of linked data (first 5 rows, first 5 columns):")
            print(linked_data.iloc[:5, :5])
            
            # Check if trait column exists and is not all NaN
            if trait in linked_data.columns and not linked_data[trait].isna().all():
                # Handle missing values
                linked_data = handle_missing_values(linked_data, trait)
                print(f"Linked data shape after handling missing values: {linked_data.shape}")
                
                if linked_data.shape[0] > 0:
                    # Check for bias in trait and demographic features
                    is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
                    
                    # Validate the data quality and save cohort info
                    note = "Dataset contains gene expression data from thyroid cancer samples with tumor type information."
                    is_usable = validate_and_save_cohort_info(
                        is_final=True,
                        cohort=cohort,
                        info_path=json_path,
                        is_gene_available=is_gene_available,
                        is_trait_available=is_trait_available,
                        is_biased=is_biased,
                        df=linked_data,
                        note=note
                    )
                    
                    # Save the linked data if it's usable
                    if is_usable:
                        os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
                        linked_data.to_csv(out_data_file)
                        print(f"Linked data saved to {out_data_file}")
                    else:
                        print("Data not usable for the trait study - not saving final linked data.")
                else:
                    print("After handling missing values, no samples remain.")
                    validate_and_save_cohort_info(
                        is_final=True,
                        cohort=cohort,
                        info_path=json_path,
                        is_gene_available=is_gene_available,
                        is_trait_available=is_trait_available,
                        is_biased=True,
                        df=pd.DataFrame(),
                        note="No valid samples after handling missing values."
                    )
            else:
                print(f"Trait column '{trait}' missing or all values are NaN.")
                validate_and_save_cohort_info(
                    is_final=True,
                    cohort=cohort,
                    info_path=json_path,
                    is_gene_available=is_gene_available,
                    is_trait_available=False,
                    is_biased=True,
                    df=linked_data,
                    note=f"Trait column '{trait}' missing or contains only NaN values."
                )
        else:
            print("No common samples found between gene expression and clinical data.")
            validate_and_save_cohort_info(
                is_final=True,
                cohort=cohort,
                info_path=json_path,
                is_gene_available=is_gene_available,
                is_trait_available=is_trait_available,
                is_biased=True,
                df=pd.DataFrame(),
                note="No common samples between gene expression and clinical data."
            )
    except Exception as e:
        print(f"Error linking or processing data: {e}")
        import traceback
        traceback.print_exc()
        validate_and_save_cohort_info(
            is_final=True,
            cohort=cohort


### Step 8: Data Normalization and Linking

In [ ]:
```python
# 1. Normalize gene symbols in the gene expression data
try:
    # First reload the gene expression data from the matrix file
    soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
    gene_data = get_genetic_data(matrix_file)
    print(f"Reloaded gene data shape: {gene_data.shape}")
    
    # We need the gene annotation for mapping
    gene_annotation = get_gene_annotation(soft_file)
    mapping_df = get_gene_mapping(gene_annotation, 'ID', 'Symbol')
    gene_data = apply_gene_mapping(gene_data, mapping_df)
    print(f"Mapped gene data shape: {gene_data.shape}")
    
    # Make sure the directory exists
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    
    # Apply normalization to gene symbols
    normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
    print(f"Gene data shape after normalization: {normalized_gene_data.shape}")
    
    # Save the normalized gene data
    normalized_gene_data.to_csv(out_gene_data_file)
    print(f"Normalized gene data saved to {out_gene_data_file}")
    
    # Use the normalized data for further processing
    gene_data = normalized_gene_data
    is_gene_available = True
except Exception as e:
    print(f"Error normalizing gene data: {e}")
    is_gene_available = False

# Redefine conversion functions from Step 2
def convert_trait(value):
    """Convert thyroid cancer status to binary: 1 for cancer, 0 for non-neoplastic."""
    if value is None:
        return None
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    if 'Non-neoplastic' in value:
        return 0  # Control
    elif 'carcinoma' in value or 'Thyroid_carcinoma' in value:
        return 1  # Case
    return None

def convert_age(value):
    """Convert age to continuous numeric value."""
    if value is None:
        return None
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    try:
        return float(value)
    except ValueError:
        return None

def convert_gender(value):
    """Convert gender to binary: 0 for female, 1 for male."""
    if value is None:
        return None
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    if value.upper() == 'F':
        return 0  # Female
    elif value.upper() == 'M':
        return 1  # Male
    return None

# 2. Load clinical data and extract features
try:
    # Load the clinical data from file
    background_info, clinical_data = get_background_and_clinical_data(matrix_file)
    
    # Extract clinical features with CORRECT indices from Step 2
    clinical_features = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=0,       # Disease status (Thyroid_carcinoma vs Non-neoplastic)
        convert_trait=convert_trait,
        gender_row=3,      # Sex row
        convert_gender=convert_gender,
        age_row=2,         # Age row
        convert_age=convert_age
    )
    
    print(f"Extracted clinical data shape: {clinical_features.shape}")
    print("Preview of clinical data (first 5 samples):")
    print(clinical_features.iloc[:, :5])
    
    # Save the properly extracted clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_features.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")
    
    is_trait_available = True
except Exception as e:
    print(f"Error extracting clinical data: {e}")
    is_trait_available = False

# 3. Link clinical and genetic data if both are available
if is_trait_available and is_gene_available:
    try:
        # Check for common sample IDs
        common_samples = set(gene_data.columns).intersection(clinical_features.columns)
        print(f"Found {len(common_samples)} common samples between gene and clinical data")
        
        if len(common_samples) > 0:
            # Link the clinical and genetic data
            linked_data = geo_link_clinical_genetic_data(clinical_features, gene_data)
            print(f"Initial linked data shape: {linked_data.shape}")
            
            # Debug the trait values before handling missing values
            print("Preview of linked data (first 5 rows, first 5 columns):")
            print(linked_data.iloc[:5, :5])
            
            # Handle missing values
            linked_data = handle_missing_values(linked_data, trait)
            print(f"Linked data shape after handling missing values: {linked_data.shape}")
            
            if linked_data.shape[0] > 0:
                # Check for bias in trait and demographic features
                is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
                
                # Validate the data quality and save cohort info
                note = "Dataset contains gene expression data from thyroid cancer samples with tumor type information."
                is_usable = validate_and_save_cohort_info(
                    is_final=True,
                    cohort=cohort,
                    info_path=json_path,
                    is_gene_available=is_gene_available,
                    is_trait_available=is_trait_available,
                    is_biased=is_biased,
                    df=linked_data,
                    note=note
                )
                
                # Save the linked data if it's usable
                if is_usable:
                    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
                    linked_data.to_csv(out_data_file)
                    print(f"Linked data saved to {out_data_file}")
                else:
                    print("Data not usable for the trait study - not saving final linked data.")
            else:
                print("After handling missing values, no samples remain.")
                validate_and_save_cohort_info(
                    is_final=True,
                    cohort=cohort,
                    info_path=json_path,
                    is_gene_available=is_gene_available,
                    is_trait_available=is_trait_available,
                    is_biased=True,
                    df=pd.DataFrame(),
                    note="No valid samples after handling missing values."
                )
        else:
            print("No common samples found between gene expression and clinical data.")
            validate_and_save_cohort_info(
                is_final=True,
                cohort=cohort,
                info_path=json_path,
                is_gene_available=is_gene_available,
                is_trait_available=is_trait_available,
                is_biased=True,
                df=pd.DataFrame(),
                note="No common samples between gene expression and clinical data."
            )
    except Exception as e:
        print(f"Error linking or processing data: {e}")
        import traceback
        traceback.print_exc()
        validate_and_save_cohort_info(
            is_final=True,
            cohort=cohort,
            info_path=json_path,
            is_gene_available=is_gene_available,
            is_trait_available=is_trait_available,
            is_biased=True,  # Assume biased if there's an error
            df=pd.DataFrame(),  # Empty dataframe for metadata
            note=f"Error in data processing: {str(e)}"


### Step 9: Data Normalization and Linking

In [ ]:
# 1. Normalize gene symbols in the gene expression data
try:
    # Make sure the directory exists
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    
    # Use the gene_data variable from the previous step (don't try to load it from file)
    print(f"Gene data shape before normalization: {gene_data.shape}")
    
    # Apply normalization to gene symbols
    normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
    print(f"Gene data shape after normalization: {normalized_gene_data.shape}")
    
    # Save the normalized gene data
    normalized_gene_data.to_csv(out_gene_data_file)
    print(f"Normalized gene data saved to {out_gene_data_file}")
    
    # Use the normalized data for further processing
    gene_data = normalized_gene_data
    is_gene_available = True
except Exception as e:
    print(f"Error normalizing gene data: {e}")
    is_gene_available = False

# 2. Load clinical data
try:
    # Load the clinical data from file
    soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
    background_info, clinical_data = get_background_and_clinical_data(matrix_file)
    
    # Debug the trait and gender conversion functions
    print("Sample conversion results:")
    sample_trait = list(clinical_data.iloc[2])[1]  # Get a sample from trait row (row 2)
    sample_gender = list(clinical_data.iloc[0])[1]  # Get a sample from gender row (row 0)
    print(f"Sample trait value: '{sample_trait}' -> {convert_trait(sample_trait)}")
    print(f"Sample gender value: '{sample_gender}' -> {convert_gender(sample_gender)}")
    
    # Extract clinical features with CORRECT indices from Step 2
    clinical_features = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=2,       # Correct trait row from Step 2
        convert_trait=convert_trait,
        gender_row=0,      # Correct gender row from Step 2
        convert_gender=convert_gender,
        age_row=None,      # Age row is None as per Step 2
        convert_age=None
    )
    
    print(f"Extracted clinical data shape: {clinical_features.shape}")
    print("Preview of clinical data (first 5 samples):")
    print(clinical_features.iloc[:, :5])
    
    # Save the properly extracted clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_features.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")
    
    is_trait_available = True
except Exception as e:
    print(f"Error extracting clinical data: {e}")
    is_trait_available = False

# 3. Link clinical and genetic data if both are available
if is_trait_available and is_gene_available:
    try:
        # Debug the column names to ensure they match
        print(f"Gene data columns (first 5): {gene_data.columns[:5].tolist()}")
        print(f"Clinical data columns (first 5): {clinical_features.columns[:5].tolist()}")
        
        # Check for common sample IDs
        common_samples = set(gene_data.columns).intersection(clinical_features.columns)
        print(f"Found {len(common_samples)} common samples between gene and clinical data")
        
        if len(common_samples) > 0:
            # Link the clinical and genetic data
            linked_data = geo_link_clinical_genetic_data(clinical_features, gene_data)
            print(f"Initial linked data shape: {linked_data.shape}")
            
            # Debug the trait values before handling missing values
            print("Preview of linked data (first 5 rows, first 5 columns):")
            print(linked_data.iloc[:5, :5])
            
            # Handle missing values
            linked_data = handle_missing_values(linked_data, trait)
            print(f"Linked data shape after handling missing values: {linked_data.shape}")
            
            if linked_data.shape[0] > 0:
                # Check for bias in trait and demographic features
                is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
                
                # Validate the data quality and save cohort info
                note = "Dataset contains gene expression data from thyroid cancer samples with tumor type information."
                is_usable = validate_and_save_cohort_info(
                    is_final=True,
                    cohort=cohort,
                    info_path=json_path,
                    is_gene_available=is_gene_available,
                    is_trait_available=is_trait_available,
                    is_biased=is_biased,
                    df=linked_data,
                    note=note
                )
                
                # Save the linked data if it's usable
                if is_usable:
                    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
                    linked_data.to_csv(out_data_file)
                    print(f"Linked data saved to {out_data_file}")
                else:
                    print("Data not usable for the trait study - not saving final linked data.")
            else:
                print("After handling missing values, no samples remain.")
                validate_and_save_cohort_info(
                    is_final=True,
                    cohort=cohort,
                    info_path=json_path,
                    is_gene_available=is_gene_available,
                    is_trait_available=is_trait_available,
                    is_biased=True,
                    df=pd.DataFrame(),
                    note="No valid samples after handling missing values."
                )
        else:
            print("No common samples found between gene expression and clinical data.")
            validate_and_save_cohort_info(
                is_final=True,
                cohort=cohort,
                info_path=json_path,
                is_gene_available=is_gene_available,
                is_trait_available=is_trait_available,
                is_biased=True,
                df=pd.DataFrame(),
                note="No common samples between gene expression and clinical data."
            )
    except Exception as e:
        print(f"Error linking or processing data: {e}")
        validate_and_save_cohort_info(
            is_final=True,
            cohort=cohort,
            info_path=json_path,
            is_gene_available=is_gene_available,
            is_trait_available=is_trait_available,
            is_biased=True,  # Assume biased if there's an error
            df=pd.DataFrame(),  # Empty dataframe for metadata
            note=f"Error in data processing: {str(e)}"
        )
else:
    # We can't proceed with linking if either trait or gene data is missing
    print("Cannot proceed with data linking due to missing trait or gene data.")
    validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=is_gene_available,
        is_trait_available=is_trait_available,
        is_biased=True,  # Data is unusable if we're missing components
        df=pd.DataFrame(),  # Empty dataframe for metadata
        note="Missing essential data components for linking (trait data or gene expression data)."
    )